<a href="https://colab.research.google.com/github/Dhareey/MNIST_OBJ_WITH_PYTORCH/blob/main/MNIST_OBJECT_RECOGNITION_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import torch
from torch import nn
import numpy as np
# Dataset to be used is from torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
  print('GPU ready, boss... Good to go...')
else:
  print('Sorry boss, u gonna have to do CPU...')

GPU ready, boss... Good to go...


In [3]:
# Prepare the data to be used
num_workers = 0
batch_size = 20
valid_size = 0.2
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = datasets.CIFAR10('data', 
                              train=True,
                              download = True,
                              transform = transform)
test_data = datasets.CIFAR10('data',
                             train = False,
                             download= True,
                             transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Get validaton data from train data
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [6]:
# Prepare dataloaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=batch_size, sampler = train_sampler, num_workers= num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, sampler= valid_sampler, num_workers= num_workers)
test_loader = DataLoader(test_data, batch_size = batch_size,num_workers= num_workers)

In [7]:
# Label Classes
classes = ['airplane', 'automobile','bird', 'cat','deer','dog','frog', 'horse', 'ship','truck']

In [8]:
# Define the Network Architecture
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    #Define the covnets layer
    self.covnet1 = nn.Conv2d(3, 16, 3, padding=1)
    self.covnet2 = nn.Conv2d(16, 32,3, padding=1)
    self.covnet3 = nn.Conv2d(32, 64, 3, padding=1)
    # Define the Fully connected layers
    self.fc1 = nn.Linear(64*4*4, 500)
    self.fc2 = nn.Linear(500,10)
    # Define the Dropouts
    self.dropout = nn.Dropout(0.2)
    # Define the maxpooling
    self.mPool = nn.MaxPool2d(2,2)
  
  #Define the forward
  def forward(self, x):
    #print('Before covnet1, xshape=', x.shape)
    x = self.mPool(F.relu(self.covnet1(x)))
    #print('After covnet1, xshape=', x.shape)
    x = self.mPool(F.relu(self.covnet2(x)))
    #print('After covnet2, xshape=', x.shape)
    x= self.mPool(F.relu(self.covnet3(x)))
    #print('After covnet3, xshape=', x.shape)
    x = x.view(-1, 64*4*4)
    #print('After flattening, xshape=', x.shape)
    x = self.dropout(x)
    #print('After dropout, xshape=', x.shape)
    x = F.relu(self.fc1(x))
    #print('After fc1, xshape=', x.shape)
    x = self.dropout(x)
    #print('After dropout2, xshape=', x.shape)
    x = self.fc2(x)
    #print('After fc2, xshape=', x.shape)
    return x

In [9]:
# Instantiate the model
model = Net()
print(model)

Net(
  (covnet1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (covnet2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (covnet3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (mPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [10]:
# Move model to gpu
if train_on_gpu:
  model.cuda()
  print('Model moved to GPU, boss...')

Model moved to GPU, boss...


In [11]:
# Set Model Optimizer and Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [24]:
# Training the model... Hold your breathe, Dare... This part could get messy
n_epochs = 100
valid_loss_min = np.Inf
for epoch in range(1,n_epochs+1):
  train_loss = 0.0
  valid_loss = 0.0
  # Set the model to train
  model.train()
  for data, target in train_loader:
    # Move data to GPU
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
      #print('Training Data Moved to GPU, bro....')
    # Clear optimizer
    optimizer.zero_grad()
    # Run the forward pass
    output = model(data)
    # Calculate the loss
    loss = criterion(output, target)
    # DO backward Propagation
    loss.backward()
    # Update Optimisation
    optimizer.step()
    #Update training Loss
    train_loss += loss.item() * data.size(0)
  
  ##########################
  #Validate the model
  ##########################
  # Set model to evaluate
  model.eval()
  for data, target in valid_loader:
    # Moved data to cuda
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
      #print('validation data Moved to GPu bro...')
    # Run forward pass
    output = model(data)
    # Calculate the loss
    loss = criterion(output, target)
    # Calc Validation Loss
    valid_loss+= loss.item()* data.size(0)

  # Calculate Average Losses
  train_loss = train_loss/len(train_loader.sampler)
  valid_loss = valid_loss/len(valid_loader.sampler)
  print('Epoch: {}\t Training Loss: {:.6f}\t Validation Loss:{:.6f}'.format(epoch, train_loss,valid_loss))

  #Save model if model is better than previous
  if valid_loss <= valid_loss_min:
    print('Validation Loss Decreased from {:.6f} to {:.6f}. Saving Model...'.format(valid_loss_min, valid_loss))
    torch.save(model.state_dict(),'model_cifar.pt')
    valid_loss_min = valid_loss


Epoch: 1	 Training Loss: 0.645497	 Validation Loss:0.806181
Validation Loss Decreased from inf to 0.806181. Saving Model...
Epoch: 2	 Training Loss: 0.637656	 Validation Loss:0.799135
Validation Loss Decreased from 0.806181 to 0.799135. Saving Model...
Epoch: 3	 Training Loss: 0.631088	 Validation Loss:0.804042
Epoch: 4	 Training Loss: 0.630410	 Validation Loss:0.792710
Validation Loss Decreased from 0.799135 to 0.792710. Saving Model...
Epoch: 5	 Training Loss: 0.623073	 Validation Loss:0.802110
Epoch: 6	 Training Loss: 0.619559	 Validation Loss:0.792776
Epoch: 7	 Training Loss: 0.611428	 Validation Loss:0.799146
Epoch: 8	 Training Loss: 0.603045	 Validation Loss:0.798630
Epoch: 9	 Training Loss: 0.598675	 Validation Loss:0.795339
Epoch: 10	 Training Loss: 0.597919	 Validation Loss:0.795425
Epoch: 11	 Training Loss: 0.586808	 Validation Loss:0.794617
Epoch: 12	 Training Loss: 0.587282	 Validation Loss:0.791163
Validation Loss Decreased from 0.792710 to 0.791163. Saving Model...
Epoch:

In [25]:
# Looks like the model will be better if it trains longer but im testing it anyway
# Load the saved model

In [26]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

In [27]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
# Set the model to evaluate
model.eval()
for data, target in test_loader:
  if train_on_gpu:
    data, target = data.cuda(), target.cuda()
  output = model(data)
  loss = criterion(output, target)
  test_loss += loss.item() * data.size(0)
  _, pred = torch.max(output, 1)
  correct_tensor = pred.eq(target.data.view_as(pred))
  correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
  for i in range(batch_size):
    label = target.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

In [28]:
# Calculating Average Test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

Test Loss: 0.771920



In [29]:
# Test Loss = 0.95
# 
for i in range(len(classes)):
  if class_total[i]> 0:
    print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (classes[i], 100*class_correct[i]/class_total[i],np.sum(class_correct[i]),np.sum(class_total[i])))
  else:
    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
print('\n Test Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 81% (812/1000)
Test Accuracy of automobile: 85% (854/1000)
Test Accuracy of  bird: 61% (617/1000)
Test Accuracy of   cat: 52% (528/1000)
Test Accuracy of  deer: 63% (634/1000)
Test Accuracy of   dog: 63% (632/1000)
Test Accuracy of  frog: 82% (824/1000)
Test Accuracy of horse: 79% (790/1000)
Test Accuracy of  ship: 85% (852/1000)
Test Accuracy of truck: 79% (794/1000)

 Test Accuracy (Overall): 73% (7337/10000)
